In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

In [1]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.1 MB/s eta 0:00:00


In [2]:
!pip install replicate
!pip install unitxt
!pip install openai
!pip install litellm
!pip install diskcache
!pip install tenacity
!pip install tabulate
!pip install git+https://github.com/ibm-granite-community/utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fs

In [5]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-znso_6bt
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-znso_6bt
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB

In [1]:
import replicate
import json
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate
import nest_asyncio
nest_asyncio.apply()
from typing import Dict, List, Optional, Callable

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

print('✅ Setup complete!')

REPLICATE_API_TOKEN loaded from Google Colab secret.
✅ Setup complete!


In [2]:
PRODUCTS = [
    {"id": "P010", "name": "Wireless Headphones", "category": "Electronics", "description": "High-quality wireless headphones with noise cancellation", "price": 149.99, "stock": 75, "delivery": {"shipping_cost": 4.99, "free_shipping_threshold": 100, "estimated_days": "2-3"}},
    {"id": "P022", "name": "Running Shoes", "category": "Sports", "description": "Lightweight running shoes with cushioned soles", "price": 89.99, "stock": 120, "delivery": {"shipping_cost": 5.99, "free_shipping_threshold": 100, "estimated_days": "3-5"}},
    {"id": "P035", "name": "Coffee Maker", "category": "Kitchen", "description": "Programmable coffee maker with 12-cup capacity", "price": 79.99, "stock": 45, "delivery": {"shipping_cost": 7.99, "free_shipping_threshold": 100, "estimated_days": "2-4"}},
    {"id": "P012", "name": "Laptop Stand", "category": "Electronics", "description": "Adjustable aluminum laptop stand with cooling", "price": 29.99, "stock": 150, "delivery": {"shipping_cost": 3.99, "free_shipping_threshold": 50, "estimated_days": "1-2"}},
    {"id": "P007", "name": "Yoga Mat", "category": "Sports", "description": "Non-slip yoga mat with carrying strap", "price": 24.99, "stock": 95, "delivery": {"shipping_cost": 4.99, "free_shipping_threshold": 50, "estimated_days": "2-3"}},
    {"id": "P045", "name": "Wireless Mouse", "category": "Electronics", "description": "Ergonomic Wireless Mouse with Rechargable Battery", "price": 49.99, "stock": 18, "delivery": {"shipping_cost": 4.99, "free_shipping_threshold": 50, "estimated_days": "2-3"}},
    {"id": "P050", "name": "Smart Watch", "category": "Electronics", "description": "Fitness tracking smart watch with heart rate monitor", "price": 199.99, "stock": 60, "delivery": {"shipping_cost": 6.99, "free_shipping_threshold": 150, "estimated_days": "2-4"}},
    {"id": "P060", "name": "Bluetooth Speaker", "category": "Electronics", "description": "Portable Bluetooth speaker with deep bass", "price": 59.99, "stock": 80, "delivery": {"shipping_cost": 4.99, "free_shipping_threshold": 100, "estimated_days": "2-3"}},
    {"id": "P070", "name": "Electric Kettle", "category": "Kitchen", "description": "1.7L electric kettle with auto shut-off", "price": 39.99, "stock": 110, "delivery": {"shipping_cost": 5.99, "free_shipping_threshold": 80, "estimated_days": "2-3"}},
    {"id": "P080", "name": "Backpack", "category": "Accessories", "description": "Water-resistant backpack with laptop compartment", "price": 54.99, "stock": 130, "delivery": {"shipping_cost": 3.99, "free_shipping_threshold": 75, "estimated_days": "2-4"}},
    {"id": "P090", "name": "Desk Lamp", "category": "Home", "description": "LED desk lamp with adjustable brightness", "price": 34.99, "stock": 90, "delivery": {"shipping_cost": 2.99, "free_shipping_threshold": 50, "estimated_days": "1-2"}},
    {"id": "P100", "name": "Gaming Keyboard", "category": "Electronics", "description": "Mechanical gaming keyboard with RGB lighting", "price": 89.99, "stock": 40, "delivery": {"shipping_cost": 5.99, "free_shipping_threshold": 100, "estimated_days": "2-3"}},
    {"id": "P110", "name": "Water Bottle", "category": "Sports", "description": "Insulated stainless steel water bottle", "price": 19.99, "stock": 200, "delivery": {"shipping_cost": 2.99, "free_shipping_threshold": 40, "estimated_days": "1-2"}},
    {"id": "P120", "name": "Sunglasses", "category": "Accessories", "description": "UV-protection polarized sunglasses", "price": 29.99, "stock": 85, "delivery": {"shipping_cost": 3.99, "free_shipping_threshold": 60, "estimated_days": "2-3"}},
    {"id": "P130", "name": "Fitness Tracker", "category": "Electronics", "description": "Activity and sleep tracking wristband", "price": 69.99, "stock": 55, "delivery": {"shipping_cost": 4.99, "free_shipping_threshold": 100, "estimated_days": "2-3"}},
    {"id": "P140", "name": "Portable Charger", "category": "Electronics", "description": "10000mAh portable power bank", "price": 39.99, "stock": 140, "delivery": {"shipping_cost": 3.99, "free_shipping_threshold": 60, "estimated_days": "2-3"}},
]

In [3]:
TOOL_REGISTRY = {}

def tool(func):
    """Decorator to register a function as a tool."""
    TOOL_REGISTRY[func.__name__] = func
    return func

def _find_product(product_id: str) -> Optional[Dict]:
    for product in PRODUCTS:
        if product['id'] == product_id:
            return product
    return None

@tool
def get_product_info(product_id: str) -> str:
    """Get product description and category."""
    product = _find_product(product_id)
    if not product:
        return "Product not found."
    return f"Product: {product['name']}\nCategory: {product['category']}\nDescription: {product['description']}"

@tool
def get_price(product_id: str) -> str:
    """Get product price information."""
    product = _find_product(product_id)
    if not product:
        return "Product not found."
    return f"Price: ${product['price']:.2f}"

@tool
def get_delivery_info(product_id: str) -> str:
    """Get delivery details."""
    product = _find_product(product_id)
    if not product:
        return "Product not found."
    delivery = product['delivery']
    free_shipping_msg = f"\nFree shipping on orders over ${delivery['free_shipping_threshold']:.2f}"
    return (f"Shipping Cost: ${delivery['shipping_cost']:.2f}\n"
            f"Estimated Delivery: {delivery['estimated_days']} business days"
            f"{free_shipping_msg}")

@tool
def check_stock(product_id: str) -> str:
    """Check stock availability."""
    product = _find_product(product_id)
    if not product:
        return "Product not found."
    stock = product['stock']
    if stock > 100:
        status = "High Stock"
    elif stock > 50:
        status = "Good Stock"
    elif stock > 10:
        status = "Limited Stock"
    else:
        status = "Low Stock"
    return f"Stock Status: {status} ({stock} units available)"

In [18]:
def agent_function_call(user_query: str) -> Dict:
    """
    Simulate agent function-calling: parse intent, select tool, call tool, return response.
    Returns: dict with 'tool_used', 'tool_args', 'response'
    """
    # For lab: simple keyword matching (participants can improve this)
    user_query_lc = user_query.lower()
    if "price" in user_query_lc:
        tool_name = "get_price"
        args = {"product_id": "P045"}
    elif "stock" in user_query_lc or "available" in user_query_lc:
        tool_name = "check_stock"
        args = {"product_id": "P045"}
    elif "delivery" in user_query_lc or "shipping" in user_query_lc:
        tool_name = "get_delivery_info"
        args = {"product_id": "P045"}
    else:
        tool_name = "get_product_info"
        args = {"product_id": "P045"}
    response = TOOL_REGISTRY[tool_name](**args)
    return {"tool_used": tool_name, "tool_args": args, "response": response}

In [16]:
def evaluate_tool_invocation(user_query: str, tool_used: str) -> dict:
    """
    Evaluate if the correct tool was selected for the user query.
    Uses Granite model for judgment.
    """
    prompt = f"""<think>
User Query: {user_query}
Tool Invoked: {tool_used}
Was this the correct tool for the query? Rate 1-5 and explain.
Return JSON: {{"score": <1-5>, "explanation": "<your brief explanation>"}}
</think>"""
    try:
        result = model.predict(prompt, max_new_tokens=200)
        return json.loads(result.strip())
    except Exception as e:
        print(f"Error: {e}")
        return {"score": 0, "explanation": "Evaluation failed"}


In [19]:
def run_lab():
    print("\nWelcome to the IBM Granite Agent Evaluation Lab!")
    print("Type a user query about a product (e.g., 'What is the price of the wireless mouse?')\n")
    user_query = input("Enter a user query: ")
    agent_result = agent_function_call(user_query)
    print(f"\nTool Used: {agent_result['tool_used']}")
    print(f"Agent Response: {agent_result['response']}")

    tool_eval = evaluate_tool_invocation(user_query, agent_result['tool_used'])
    print("\nTool Invocation Evaluation:", tool_eval)

    response_eval = evaluate_final_response(user_query, agent_result['response'])
    print("Final Response Evaluation:", response_eval)

if __name__ == "__main__":
    run_lab()


Welcome to the IBM Granite Agent Evaluation Lab!
Type a user query about a product (e.g., 'What is the price of the wireless mouse?')

Enter a user query: I'm looking for an ergonomic wireless mouse

Tool Used: get_product_info
Agent Response: Product: Wireless Mouse
Category: Electronics
Description: Ergonomic Wireless Mouse with Rechargable Battery


/tmp/ipython-input-16-1104936071.py:13: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  result = model.predict(prompt, max_new_tokens=200)


Error: Expecting value: line 1 column 1 (char 0)

Tool Invocation Evaluation: {'score': 0, 'explanation': 'Evaluation failed'}
Error: Expecting value: line 1 column 1 (char 0)
Final Response Evaluation: {'score': 0, 'explanation': 'Evaluation failed'}


In [6]:
!pip install databricks-agents dbruntime
!dbutils.library.restartPython()

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [14]:
import mlflow
import pandas as pd
import replicate
import json
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

def try_agent(example_name: str, user_input: str):
    with mlflow.start_run(run_name=example_name):
        mlflow.log_param("user_input", user_input)

        # Simulate agent response
        response = try_agent(user_input)

        mlflow.log_param("agent_response", response)
        print(f"\n{example_name}")
        print("Input:", user_input)
        print("Response:", response)

        # Optionally log output to a text file
        with open(f"{example_name.replace(' ', '_')}_output.txt", "w") as f:
            f.write(f"Input: {user_input}\nResponse: {response}")
        mlflow.log_artifact(f"{example_name.replace(' ', '_')}_output.txt")


response_eval =  pd.DataFrame({
    "request": [
      {
      # Recommended `messages` format
        "messages": [{
          "role": "user",
          "content": "I'm looking for an ergonimic wireless mouse"
        }],
      },
      # SplitChatMessagesRequest format
      {
        "query": "Is the product available in stock?",
        "history": [
          {"role": "user", "content": "I'm looking for an ergonimic wireless mouse"},
          {"role": "agent", "content": "Product P045 is a wireless mouse with ergonomic support and programmable buttons"},

        ],
      }
      # Note: Using a primitive string is discouraged. The string will be wrapped in the
      # OpenAI messages format before being passed to your agent.
    ],
    "response": [
        "Product P045 is a wireless mouse with ergonomic support and programmable buttons.",
        "Product P045 is currently available in your selected region",
    ],
    "expected_response": [ # Optional, needed for judging correctness.
        "Product P045 is a wireless mouse with ergonomic support and programmable buttons",
        "Product P045 is currently available in your selected region"
    ]
}
)

global_guidelines = {
  "english": ["The response must be in English"],
  "clarity": ["The response must be clear, coherent, and concise"],
}

#evaluate_final_response(user_query, agent_result)


# with mlflow.start_run() as run:
#   system_prompt = "Answer the following question in two sentences"
#   basic_qa_model = mlflow.openai.log_model(
#       model=model,
#       name="model",
#       messages=[
#           {"role": "system", "content": system_prompt},
#           {"role": "user", "content": "{question}"},
#       ],
#   )
model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)



results = mlflow.evaluate(
      model,
      response_eval,
      # targets="ground_truth",  # specify which column corresponds to the expected output
      model_type="question-answering",  # model type indicates which metrics are relevant for this task
      evaluators="default",
)
results.metrics

2025/07/05 08:33:58 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/07/05 08:33:58 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
/usr/local/lib/python3.11/dist-packages/mlflow/models/evaluation/base.py:1069: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  return fn(model_input)


ValueError: Argument `prompt` is expected to be a string. Instead found <class 'pandas.core.frame.DataFrame'>. If you want to run the LLM on multiple prompts, use `generate` instead.

In [15]:
import mlflow
import pandas as pd
import replicate
import json
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

def try_agent(example_name: str, user_input: str):
    with mlflow.start_run(run_name=example_name):
        mlflow.log_param("user_input", user_input)

        # Simulate agent response
        response = try_agent(user_input)

        mlflow.log_param("agent_response", response)
        print(f"\n{example_name}")
        print("Input:", user_input)
        print("Response:", response)

        # Optionally log output to a text file
        with open(f"{example_name.replace(' ', '_')}_output.txt", "w") as f:
            f.write(f"Input: {user_input}\nResponse: {response}")
        mlflow.log_artifact(f"{example_name.replace(' ', '_')}_output.txt")

# Create evaluation data with proper structure for MLflow
response_eval = pd.DataFrame({
    "inputs": [
        "I'm looking for an ergonimic wireless mouse",
        "Is the product available in stock?"
    ],
    "targets": [  # Changed from "expected_response" to "targets"
        "Product P045 is a wireless mouse with ergonomic support and programmable buttons",
        "Product P045 is currently available in your selected region"
    ]
})

global_guidelines = {
    "english": ["The response must be in English"],
    "clarity": ["The response must be clear, coherent, and concise"],
}

# Create a wrapper class to handle the model interface properly
class ReplicateModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, inputs):
        """MLflow expects a predict method that takes inputs and returns predictions"""
        if isinstance(inputs, pd.DataFrame):
            # Handle DataFrame input
            if 'inputs' in inputs.columns:
                prompts = inputs['inputs'].tolist()
            else:
                # If no 'inputs' column, use the first column
                prompts = inputs.iloc[:, 0].tolist()
        elif isinstance(inputs, list):
            prompts = inputs
        else:
            prompts = [str(inputs)]

        predictions = []
        for prompt in prompts:
            try:
                # Use invoke instead of __call__ to avoid deprecation warning
                response = self.model.invoke(prompt)
                predictions.append(response)
            except Exception as e:
                print(f"Error generating response for prompt '{prompt}': {e}")
                predictions.append(f"Error: {str(e)}")

        return predictions

# Initialize the model
model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens": 1024, "temperature": 0.2},
)

# Wrap the model
wrapped_model = ReplicateModelWrapper(model)

# Run evaluation
results = mlflow.evaluate(
    wrapped_model,
    response_eval,
    targets="targets",  # specify which column corresponds to the expected output
    model_type="question-answering",  # model type indicates which metrics are relevant for this task
    evaluators="default",
)

print("Evaluation Results:")
print(results.metrics)

MlflowException: The model argument must be a string URI referring to an MLflow model, an MLflow Deployments endpoint URI, an instance of `mlflow.pyfunc.PyFuncModel`, a function, or None.

In [16]:
import mlflow
import pandas as pd
import replicate
import json
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

def try_agent(example_name: str, user_input: str):
    with mlflow.start_run(run_name=example_name):
        mlflow.log_param("user_input", user_input)

        # Simulate agent response
        response = try_agent(user_input)

        mlflow.log_param("agent_response", response)
        print(f"\n{example_name}")
        print("Input:", user_input)
        print("Response:", response)

        # Optionally log output to a text file
        with open(f"{example_name.replace(' ', '_')}_output.txt", "w") as f:
            f.write(f"Input: {user_input}\nResponse: {response}")
        mlflow.log_artifact(f"{example_name.replace(' ', '_')}_output.txt")

# Create evaluation data with proper structure for MLflow
response_eval = pd.DataFrame({
    "inputs": [
        "I'm looking for an ergonimic wireless mouse",
        "Is the product available in stock?"
    ],
    "targets": [  # Changed from "expected_response" to "targets"
        "Product P045 is a wireless mouse with ergonomic support and programmable buttons",
        "Product P045 is currently available in your selected region"
    ]
})

global_guidelines = {
    "english": ["The response must be in English"],
    "clarity": ["The response must be clear, coherent, and concise"],
}

# Create a PyFunc model wrapper for MLflow
class ReplicatePyFuncModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model_name, api_token, model_kwargs):
        self.model_name = model_name
        self.api_token = api_token
        self.model_kwargs = model_kwargs
        self.model = None

    def load_context(self, context):
        """Load the model when needed"""
        from langchain_community.llms import Replicate
        self.model = Replicate(
            model=self.model_name,
            replicate_api_token=self.api_token,
            model_kwargs=self.model_kwargs,
        )

    def predict(self, context, model_input):
        """Predict method required by MLflow PyFunc"""
        if isinstance(model_input, pd.DataFrame):
            # Handle DataFrame input
            if 'inputs' in model_input.columns:
                prompts = model_input['inputs'].tolist()
            else:
                # If no 'inputs' column, use the first column
                prompts = model_input.iloc[:, 0].tolist()
        elif isinstance(model_input, list):
            prompts = model_input
        else:
            prompts = [str(model_input)]

        predictions = []
        for prompt in prompts:
            try:
                # Use invoke instead of __call__ to avoid deprecation warning
                response = self.model.invoke(prompt)
                predictions.append(response)
            except Exception as e:
                print(f"Error generating response for prompt '{prompt}': {e}")
                predictions.append(f"Error: {str(e)}")

        return predictions

# Initialize and log the model
pyfunc_model = ReplicatePyFuncModel(
    model_name="ibm-granite/granite-3.3-8b-instruct",
    api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens": 1024, "temperature": 0.2}
)

# Log the model to MLflow
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="replicate_model",
        python_model=pyfunc_model,
        pip_requirements=[
            "langchain-community",
            "replicate"
        ]
    )
    model_uri = f"runs:/{run.info.run_id}/replicate_model"

# Run evaluation using the logged model URI
results = mlflow.evaluate(
    model_uri,
    response_eval,
    targets="targets",  # specify which column corresponds to the expected output
    model_type="question-answering",  # model type indicates which metrics are relevant for this task
    evaluators="default",
)

print("Evaluation Results:")
print(results.metrics)

/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/07/05 08:42:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/05 08:42:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


2025/07/05 08:42:45 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/07/05 08:42:45 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.
2025/07/05 08:42:48 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in 

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
2025/07/05 08:43:26 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/07/05 08:43:26 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/07/05 08:43:26 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/07/05 08:43:26 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'ari_grade_level' at index 3 in the `extra_metrics` parameter because it returned None.
2025/07/05 08:43:28 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/07/05 08:43:28 W

Evaluation Results:
{'toxicity/v1/mean': 0.002407981148280669, 'toxicity/v1/variance': 5.033698222653233e-06, 'toxicity/v1/p90': 0.004202853528840933, 'toxicity/v1/ratio': 0.0, 'exact_match/v1': 0.0}


In [4]:
from unitxt.api import evaluate, create_dataset
from unitxt.inference import CrossProviderInferenceEngine
from unitxt.llm_as_judge import LLMJudgeDirect, DirectCriteriaCatalogEnum

from ibm_granite_community.notebook_utils import get_env_var


import nest_asyncio
nest_asyncio.apply()

In [5]:
data = [
    {"question": "I'm looking for an ergonomic Wireless Mouse"},
    {"question": "Is the product available in stock?"}
]

predictions = [
    """Product P045 is a wireless mouse with ergonomic support and programmable buttons""",
    """Product P045 is currently available in your selected region"""
]

In [11]:
metric = LLMJudgeDirect(
    evaluator_name="GRANITE3_3_8B",
    inference_engine=CrossProviderInferenceEngine(
        model="replicate/ibm-granite/granite-3.3-8b-instruct",
        provider="replicate",
        credentials={'api_token': get_env_var('REPLICATE_API_TOKEN')},
        provider_specific_args={'max_requests_per_second': 1}
    ),
    criteria=DirectCriteriaCatalogEnum.ANSWER_RELEVANCE.value,
    context_fields=["question"],
    criteria_field="criteria",
)

replicate/ibm-granite/granite-3.3-8b-instruct is not configured for provider replicate. Supported models: granite-3-2-8b-instruct,granite-vision-3-2-2b,granite-3-1-8b-instruct,granite-3-1-2b-instruct,granite-3-8b-instruct,granite-3-2b-instruct,granite-8b-code-instruct-128k,granite-20b-code-instruct-8k,llama-2-13b,llama-2-13b-chat,llama-2-70b,llama-2-70b-chat,llama-2-7b,llama-2-7b-chat,llama-3-1-405b-instruct,llama-3-70b,llama-3-70b-instruct,llama-3-8b,llama-3-8b-instruct,llama-3-3-70b-instruct,llama-4-maverick,llama-4-scout,mistral-7b-instruct-v0.2,mistral-7b-v0.1,mixtral-8x7b-instruct-v0.1,gpt-4-1. Using un normalized name will make it impossible to switch to different provider on request.


In [12]:
dataset = create_dataset(
    task="tasks.qa.open",
    test_set=data,
    metrics=[metric],
    split="test"
)

LoadFromDictionary sets 'data_classification_policy' to ['proprietary'] by default when loading from python dictionary.
To use a different value or remove this message, explicitly set the `data_classification_policy` attribute of the loader.



Generating test split: 0 examples [00:00, ? examples/s]

In [13]:
results = evaluate(predictions=predictions, data=dataset)


print("Global Scores:")
print(results.global_scores.summary)

print("Instance Scores:")
print(results.instance_scores)


replicate/ibm-granite/granite-3.3-8b-instruct is not configured for provider replicate. Supported models: granite-3-2-8b-instruct,granite-vision-3-2-2b,granite-3-1-8b-instruct,granite-3-1-2b-instruct,granite-3-8b-instruct,granite-3-2b-instruct,granite-8b-code-instruct-128k,granite-20b-code-instruct-8k,llama-2-13b,llama-2-13b-chat,llama-2-70b,llama-2-70b-chat,llama-2-7b,llama-2-7b-chat,llama-3-1-405b-instruct,llama-3-70b,llama-3-70b-instruct,llama-3-8b,llama-3-8b-instruct,llama-3-3-70b-instruct,llama-4-maverick,llama-4-scout,mistral-7b-instruct-v0.2,mistral-7b-v0.1,mixtral-8x7b-instruct-v0.1,gpt-4-1. Using un normalized name will make it impossible to switch to different provider on request.
Starting evaluation with evaluator "GRANITE3_3_8B" and provider "CrossProviderInferenceEngine
Reading criteria from self. Criteria is a single CriteriaWithOptions, replicating it for all predictions
Criteria names are 'answer_relevance'
Inferring batch 1 / 1 with 4 instances (found 0 instances in ./


LiteLLM Inference (replicate/ibm-granite/granite-3.3-8b-instruct):   0%|          | 0/4 [00:00<?, ?it/s]
LiteLLM Inference (replicate/ibm-granite/granite-3.3-8b-instruct): 100%|██████████| 4/4 [00:01<00:00,  2.15it/s]


The assessment was generated successfully.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Inferring batch 1 / 1 with 4 instances (found 0 instances in ./inference_engine_cache/CrossProviderInferenceEngine)



LiteLLM Inference (replicate/ibm-granite/granite-3.3-8b-instruct):   0%|          | 0/4 [00:00<?, ?it/s]
LiteLLM Inference (replicate/ibm-granite/granite-3.3-8b-instruct):  25%|██▌       | 1/4 [00:01<00:04,  1.40s/it]
LiteLLM Inference (replicate/ibm-granite/granite-3.3-8b-instruct): 100%|██████████| 4/4 [00:01<00:00,  2.65it/s]


The selections were calculated successfully.
Global Scores:
| score_name       |   score | ci_low   | ci_high   |
|:-----------------|--------:|:---------|:----------|
| answer_relevance |       1 |          |           |
| score            |       1 |          |           |
Main Score: answer_relevance
Num Instances: 2
Instance Scores:
[0] (dict):
    question (str):
        I'm looking for an ergonomic Wireless Mouse
    metadata (dict):
        data_classification_policy (list):
            [0] (str):
                proprietary
        num_demos (int):
            0
        demos_pool_size (int):
            0
        template (str):
            templates.qa.open
    answers (list):
    source (str):
        Answer the question.
        Question:
        I'm looking for an ergonomic Wireless Mouse
        Answer:
        
    metrics (list):
        [0] (str):
            {
                "__type__": "llm_judge_direct",
                "evaluator_name": "GRANITE3_3_8B",
          

In [14]:
results = evaluate(predictions=predictions, data=dataset)

Starting evaluation with evaluator "GRANITE3_3_8B" and provider "CrossProviderInferenceEngine
Reading criteria from self. Criteria is a single CriteriaWithOptions, replicating it for all predictions
Criteria names are 'answer_relevance'
Inferring batch 1 / 1 with 0 instances (found 4 instances in ./inference_engine_cache/CrossProviderInferenceEngine)
The assessment was generated successfully.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Inferring batch 1 / 1 with 0 instances (found 4 instances in ./inference_engine_cache/CrossProviderInferenceEngine)
The selections were calculated successfully.


In [15]:
print("Global Scores:")
print(results.global_scores.summary)

print("Instance Scores:")
print(results.instance_scores)

Global Scores:
| score_name       |   score | ci_low   | ci_high   |
|:-----------------|--------:|:---------|:----------|
| answer_relevance |       1 |          |           |
| score            |       1 |          |           |
Main Score: answer_relevance
Num Instances: 2
Instance Scores:
[0] (dict):
    question (str):
        I'm looking for an ergonomic Wireless Mouse
    metadata (dict):
        data_classification_policy (list):
            [0] (str):
                proprietary
        num_demos (int):
            0
        demos_pool_size (int):
            0
        template (str):
            templates.qa.open
    answers (list):
    source (str):
        Answer the question.
        Question:
        I'm looking for an ergonomic Wireless Mouse
        Answer:
        
    metrics (list):
        [0] (str):
            {
                "__type__": "llm_judge_direct",
                "evaluator_name": "GRANITE3_3_8B",
                "inference_engine": {
                 

In [ ]:
def evaluate_response(query: str, tool_used: str, response: str) -> dict:
    """Evaluate an agent's response using the Granite model"""

    prompt = f"""<think>
Please evaluate this AI agent interaction:

User Query: {query}
Tool Used: {tool_used}
Agent Response: {response}

Rate on a scale of 1-5 (5 being best) and provide a brief explanation.
Return your evaluation in this JSON format:
{{"score": <1-5>, "explanation": "<your brief explanation>"}}
</think>"""

    try:
        # Get evaluation from Granite
        result = model.predict(prompt, max_new_tokens=200)
        return json.loads(result.strip())
    except:
        return {"score": 0, "explanation": "Evaluation failed"}

In [ ]:
results= evaluate_response(predictions)

In [ ]:
def try_agent(query: str, tool):
    with mlflow.start_run(run_name=example_name):
        mlflow.log_param("user_input", user_input)

        # Simulate agent response
        response = try_agent(user_input)

        mlflow.log_param("agent_response", response)
        print(f"\n{example_name}")
        print("Input:", user_input)
        print("Response:", response)

        # Optionally log output to a text file
        with open(f"{example_name.replace(' ', '_')}_output.txt", "w") as f:
            f.write(f"Input: {user_input}\nResponse: {response}")
        mlflow.log_artifact(f"{example_name.replace(' ', '_')}_output.txt")

In [ ]:
# Agent Evaluation tests starts here!
try_agent(
    query="I'm looking for an ergonimic wireless mouse",  # Your question
    tool=agent_tool
)

## Your Turn!

Try creating your own queries below. Here are some ideas:
- Try different calculations
- Check weather for different cities
- Analyze different text samples

Just copy and modify the example below: